<a href="https://colab.research.google.com/github/aksj98/EmpiricallyEmergent/blob/main/BERT_Probing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for probing BERT on the IMDB movie review dataset 

## Install and imports 

In [3]:
#@title Install required libraries
!pip install transformers
!pip install torch
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#@title Imports
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer, DistilBertModel, BertTokenizer, BertModel, AdamW, Trainer, TrainingArguments, PreTrainedModel
from datasets import load_dataset

## Loading dataset and pre-processing

In [5]:
#@title Load dataset
dataset = load_dataset("imdb")
train_dataset = dataset['train']
test_dataset = dataset['test']

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#@title BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
#@title DistilBERT Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [ ]:
#@title Tokenizing the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
#@title Format the dataset
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


## Modelling 

### Base-BERT

In [7]:
#@title Load pretrained Base-BERT
bert_base_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
#@title Create class for finetuning - for baseBERT
class FineTunedBERT(PreTrainedModel):
    def __init__(self, bert_base_model):
        super(FineTunedBERT, self).__init__(bert_base_model.config)
        self.bert = bert_base_model
        self.dropout = nn.Dropout(0.1)
        self.num_labels=2
        self.fc = nn.Linear(768, self.num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)

        if labels is not None:
            loss_fn = CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.num_labels), labels.view(-1))
            return loss, logits
        else:
            return logits

model = FineTunedBERT(bert_base_model)


### DistilBERT

In [ ]:
#@title Load pretrained model
distilbert_base_model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
#@title Custom finetuning class
class FineTunedDistilBERT(PreTrainedModel):
    def __init__(self, distilbert_base_model):
        super(FineTunedDistilBERT, self).__init__(distilbert_base_model.config)
        self.distilbert = distilbert_base_model
        self.dropout = nn.Dropout(0.1)
        self.num_labels = 2
        self.fc = nn.Linear(768, self.num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)

        if labels is not None:
            loss_fn = CrossEntropyLoss()
            loss = loss_fn(logits.view(-1, self.num_labels), labels.view(-1))
            return loss, logits
        else:
            return logits


In [ ]:
#@title Init model
model = FineTunedDistilBERT(distilbert_base_model)
model.config = distilbert_base_model.config

## Training

In [ ]:
#@title Training settings
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1, #only 1 epoch since I don't like watching the world burn :)
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    save_steps=1000,
    seed=42,
    learning_rate=2e-5,
)

In [ ]:
#@title Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

## Evaluation and exploration

In [ ]:
#@title Evaluation
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)